# Task 3 - Learning from the present to fill in gaps in the past

Should contain:

* Exploit data from two sources (e.g. large scale gridded analysis and in situ observations) – clearly articulating their provenance.

* Apply both supervised (e.g. discrimination, classification, or other supervised ML) and unsupervised (e.g. PCAs, Cluster analysis, or other unsupervised ML)  statistical techniques in some way in a clear,  transparent and reproducible way.

* Clear and illuminating text (approx. 2000 words) including appropriate references and 4-5 figures outlining the environmental challenge and demonstrating the insights of the statistical techniques.

Currently I think that I will use the `rain_hourly.csv` data from multiple raing gauges (close by) to reconstruct data from a single rain gauge. 
I think there are missing rain values so maybe that is the ting to do. 
Fill in the rain gauge data using a reconstruction then look at bacteria.

At first look what might be interesting is to look at the rain data from Concord, West Ryde, Homebush, Gladesville and Ashfield and reconstruct data for the missing values.

The idea could be that there are bacteria readings without rain gauge data so could see if reconstructed rain data matches up with when bacteria readings occur at Cabarita beach - that is does the reconstructed rain data match when high samples of bacteria are recorded?

In [9]:
using Pkg
Pkg.activate("..")

using DataFrames, CSV, Dates

  Activating environment at `~/Documents/GitHub/MATH5271/Project.toml`


## Rain gauge data

**Describe clearly rain data**

Have data from multiple rain gauges.
Will look to make a reconstruction of missing data using four gauges Concord, West Ryde, Homebush, Gladesville.

In [89]:
data_path = joinpath("../..", "Riverwatch/Data/raw")
rain_path = joinpath(data_path, "rain_hourly.csv")
rain_data = CSV.File(rain_path; drop = ["Lilyfield", "Fivedock"])
rain = DataFrame(rain_data);

The raw data has separate `Date` and `Time` columns so we create a `DateTime` array for ease of use then replcae the `Date` and `Time` columns with this single `DateTime` column.

In [90]:
date_format = Dates.DateFormat("d/m/Y")
new_dates = parse.(Date, rain[:, :date], date_format)
find = findfirst(new_dates .== Date(0000, 01, 01))
new_dates[1:find-1] .+= Year(1900)
new_dates[find:end] .+= Year(2000)
datetime = @. new_dates + rain[:, :time] 
rain =  rain[!, Not([:date, :time])]
insertcols!(rain, 1, :datetime => datetime)
first(rain, 5)

,datetime,Homebush,WestRyde,Concord,Gladesville,Ashfield
,DateTime,Float64?,Float64?,Float64?,Float64?,Float64?
1,1996-01-01T00:00:00,0.0,0.0,0.0,0.0,0.0
2,1996-01-01T01:00:00,0.0,0.0,0.0,0.0,0.0
3,1996-01-01T02:00:00,0.0,0.0,0.0,0.0,0.0
4,1996-01-01T03:00:00,0.0,0.0,0.0,0.0,0.0
5,1996-01-01T04:00:00,0.0,0.0,0.0,0.0,0.0


Now have a `DataFrame` with the hourly rain from each rain gauge in the area of the Parramatta river swim sites.
We can get a summary of this data to look at some summary statistics.

In [91]:
describe(rain)

,variable,mean,min,median,max,nmissing
,Symbol,Union…,Any,Union…,Any,Int64
1,datetime,,1996-01-01T00:00:00,,2021-04-01T01:00:00,0
2,Homebush,0.111786,0.0,0.0,77.0,118887
3,WestRyde,0.117512,0.0,0.0,50.5,68507
4,Concord,0.105908,0.0,0.0,46.5,3087
5,Gladesville,0.113492,0.0,0.0,53.5,1079
6,Ashfield,0.128189,0.0,0.0,62.0,121538


We can see the mean rainfall over the time period is reasonably similar (as we would expect with the rain gauges being close to each other) ranging from 0.11 (2 decimal places) at Concord to 0.13 (2 decimal places) at Ashfield.

From the summary there are also many missing values at Homebush, Fivedock and Ashfield **(find out why)**.
Using surrounding rain gauges we would now like to predict the rainfall 